# Voting
- Recvae모델에서 나온 결과값과 현재까지 가장 잘나온 모델의 결과를 hard voting 해보았습니다.

In [2]:
import pandas as pd
import pickle
from scipy import sparse
import numpy as np

In [18]:
raw_data=pd.read_csv('./data/train/train_ratings.csv')

In [3]:
result1= pd.read_csv('./code/RecVAE/recvae_dir_genre.csv')
result2= pd.read_csv('./code/output/answer/glance.csv')

# CSR로 변환

In [26]:
with open('show2id.pkl','rb') as f:
    show2id= pickle.load(f)
with open('profile2id.pkl','rb') as f:
    profile2id= pickle.load(f)

In [27]:
def numerize(tp, profile2id, show2id):
    uid = tp['user'].apply(lambda x: profile2id[x])
    sid = tp['item'].apply(lambda x: show2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [28]:
result1= numerize(result1, profile2id, show2id)
result2 = numerize(result2, profile2id, show2id)
raw_data0=  numerize(raw_data, profile2id, show2id)

In [42]:
n_users = raw_data0['uid'].max() + 1
n_items = len(raw_data0['sid'].unique())
rows, cols = result1['uid'],result1['sid']
result1= sparse.csr_matrix((np.ones_like(rows),
                                 (rows, cols)), dtype='float64',
                                 shape=(n_users, n_items))

rows, cols = result2['uid'], result2['sid']
result2 = sparse.csr_matrix((np.ones_like(rows),
                                 (rows, cols)), dtype='float64',
                                 shape=(n_users, n_items))

# csr matrix합치고 선호도 반영

In [55]:
bg= result1 + result2

In [67]:
bg.toarray()[0][604]

20000.0

In [62]:
bg

<31360x6807 sparse matrix of type '<class 'numpy.float64'>'
	with 497722 stored elements in Compressed Sparse Row format>

In [63]:
lst= np.load("./data/train/lst.npy")

In [69]:
lst.shape

(31360, 6807)

In [66]:
# lst 곱해주되 만일 값이 2이면 그 아이템은 반드시 추천 리스트에 들어가야하기 때문에 *10000 
bg[bg.toarray()==2] *=10000

In [76]:
# 각 아이템들에 해당 아이템의 장르 선호도를 곱해준다. 
bg0= bg.toarray() * lst

# 큰 값들 10개 뽑아주기

In [88]:
idx= np.argsort(-bg0)[:,:10]

In [89]:
idx

array([[ 751,  604,  299, ...,  359,  507, 2333],
       [  82,   40,  144, ...,  160,  287,  374],
       [ 264,   80,  742, ...,  130,  601,  821],
       ...,
       [ 264,   80,   84, ...,  730,  760,  159],
       [  46,  840, 2008, ...,  836, 1447,  939],
       [ 374,  202,  209, ...,  440,  474,  203]])

In [94]:
### sample_submission 형태로 바꾸고 id 다시 바꾸기
user = []
item = []
for i_idx, arr_10 in enumerate(idx):
    user.extend([i_idx]*10)
    item.extend(arr_10)

u = pd.DataFrame(user, columns=['user'])
i = pd.DataFrame(item, columns=['item'])
all = pd.concat([u, i], axis=1)

In [95]:
re_p2id = dict((v, k) for k, v in profile2id.items())
re_s2id = dict((v, k) for k, v in show2id.items())

In [96]:
def de_numerize(tp, re_p2id, re_s2id):
    uid = tp['user'].apply(lambda x: re_p2id[x])
    sid = tp['item'].apply(lambda x: re_s2id[x])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

ans = de_numerize(all, re_p2id, re_s2id)
ans.columns = ['user', 'item']
new_ans = ans.sort_values('user')
new_ans.reset_index(drop=True, inplace=True)

In [98]:
new_ans

,user,item
0,11,2407
1,11,8961
2,11,4886
3,11,3986
4,11,3156
...,...,...
313595,138493,8874
313596,138493,33615
313597,138493,1240
313598,138493,2762


In [99]:
new_ans.to_csv("/opt/ml/input/code/output/answer/brillglance.csv", index=False)